In [1]:
import json
import openai
from bs4 import BeautifulSoup as bs
import requests
import config

In [2]:
# Reading the API key from a file
service_key = config.api_key
openai.api_key = service_key

In [3]:
# Get the URL
r =requests.get('https://snyk.io/advisor/npm-package/mocha')

In [4]:
# TODO: Parse the correct URL
# with open("myresponse.html") as fp:
#     soup = bs(fp, "html.parser")
soup = bs(r.text, "html.parser")

In [5]:
# Parse the HTML for the data
data_set = []
for tag in soup.find_all('div', class_='number'):
    data_set.append(tag.text)
for tag in soup.find_all('ul', class_='scores'):
    for ele in tag.find_all('li'):
        data_set.append(ele.text)
for tag in soup.find_all('dl', class_='stats'):
    for ele in tag.find_all('div'):
        text = ele.text.replace('\n', '')
        text = " ".join(text.split())
        data_set.append(text)

In [6]:
data_set

['Package Health Score 95 / 100',
 'Popularity Key ecosystem project',
 'Maintenance Healthy',
 'Security No known security issues',
 'Community Active',
 'Dependents 0',
 'GitHub Stars 20.54K',
 'Forks 2.83K',
 'Contributors 430',
 'Open Issues 245',
 'Open PR 46',
 'Last Release 23 days ago',
 'Last Commit 2 days ago',
 'Readme.md Yes',
 'Contributing.md Yes',
 'Code of Conduct Yes',
 'Contributors 430',
 'Funding Yes',
 'Node.js Compatibility >= 10.12.0',
 'Age 10 years',
 'Dependencies 25 Direct',
 'Versions 176',
 'Install Size 3.07 MB',
 'Dist-tags 3',
 '# of Files 75',
 'Maintainers 4',
 'TS Typings Yes']

In [7]:
# Create list of JSON objects
list_of_jsons = []
for line in data_set:
    json_obj = {"text": line}
    list_of_jsons.append(json_obj)

In [8]:
# Create file with data
with open('data.jsonl', 'w') as json_file:
    for ele in list_of_jsons:
        json.dump(ele, json_file)
        json_file.write('\n')

In [9]:
# Send file to openAI
openai.File.create(file=open("data.jsonl"), purpose='answers')

Upload progress: 100%|██████████| 1.08k/1.08k [00:00<00:00, 2.31kit/s]


<File file id=file-YRzdFPIHSLqyG0bAcu6FV1nh at 0x7f79be99dc20> JSON: {
  "bytes": 836,
  "created_at": 1622324016,
  "filename": "data.jsonl",
  "id": "file-YRzdFPIHSLqyG0bAcu6FV1nh",
  "object": "file",
  "purpose": "answers",
  "status": "uploaded",
  "status_details": null
}

In [11]:
# Get an answer
openai.Answer.create(
    search_model="ada", 
    model="curie", 
    question="Can I build website with this package?", 
    file="file-YRzdFPIHSLqyG0bAcu6FV1nh", 
    examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
    examples=[["What is human life expectancy in the United States?", "78 years."]], 
    max_rerank=10,
    max_tokens=5,
    stop=["\n", "<|endoftext|>"]
)

<OpenAIObject answer at 0x7f79be93f4f0> JSON: {
  "answers": [
    "Yes, you can."
  ],
  "completion": "cmpl-350sVhuGzLDXyoPRRI9mHltHaA258",
  "file": "file-YRzdFPIHSLqyG0bAcu6FV1nh",
  "model": "curie:2020-05-03",
  "object": "answer",
  "search_model": "ada:2020-05-03",
  "selected_documents": [
    {
      "document": 0,
      "object": "search_result",
      "score": 56.659,
      "text": "Package Health Score 95 / 100"
    }
  ]
}